<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/BEC_QV_Antigravity_Sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Cell 1: Setup and Imports
!pip install numpy matplotlib scipy -q
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
from scipy.ndimage import gaussian_filter

# Configure Matplotlib
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['animation.embed_limit'] = 150 # Set limit to 50 MB

print("✅ Cell 1 Complete: Dependencies ready.")

✅ Cell 1 Complete: Dependencies ready.


In [3]:
# Cell 2: NGFT Constants and Simulation Parameters (V23 - Quantum Vortex Simulation)

# --- Simulation Domain ---
LATTICE_SIZE = 200
DT = 0.1
TOTAL_FRAMES = 500

# --- NGFT Physics Parameters ---
# How strongly the massive object curves spacetime.
GRAVITATIONAL_STRENGTH = 0.8

# How strongly the "programmed" vortices create counter-curvature. This is the key tuning parameter.
VORTEX_REPULSION_STRENGTH = 2.5

# --- Physical Objects ---
# Massive Object (e.g., a planet)
MASSIVE_OBJECT_POS = (LATTICE_SIZE // 2, LATTICE_SIZE // 2)
MASSIVE_OBJECT_DENSITY = 100.0
MASSIVE_OBJECT_RADIUS = LATTICE_SIZE / 10

# Bose-Einstein Condensate (BEC)
BEC_POS = (LATTICE_SIZE // 2, LATTICE_SIZE // 2)
BEC_RADIUS = LATTICE_SIZE / 4

# Test Particle
TEST_PARTICLE_START_POS = (LATTICE_SIZE / 4, LATTICE_SIZE / 2)

# --- Quantum Vortex Parameters ---
NUM_VORTICES = 7 # Number of vortices to create in the BEC
VORTEX_CORE_SIZE = 1.5 # How sharp the vortex is
VORTEX_CHARGE = 1 # Topological charge (+1 or -1)

print("✅ Cell 2 Complete (V23): Quantum Vortex parameters configured.")

✅ Cell 2 Complete (V23): Quantum Vortex parameters configured.


In [1]:
# Cell 3: Core Physics and Helper Functions (V23)

class TestParticle:
    """A simple particle to probe the gravitational field."""
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.vx, self.vy = 0.0, 0.0
        self.path = [(x, y)]

def initialize_gravity_field(lattice_size):
    """Creates the background gravitational potential from the massive object."""
    x, y = np.meshgrid(np.arange(lattice_size), np.arange(lattice_size))
    dist_sq = (x - MASSIVE_OBJECT_POS[0])**2 + (y - MASSIVE_OBJECT_POS[1])**2
    # A Gaussian potential well
    potential = -MASSIVE_OBJECT_DENSITY * np.exp(-dist_sq / (2 * MASSIVE_OBJECT_RADIUS**2))
    return potential

def initialize_bec_with_vortices(lattice_size):
    """Creates the BEC wave function and embeds quantum vortices."""
    x, y = np.meshgrid(np.arange(lattice_size), np.arange(lattice_size))

    # Start with a uniform BEC density
    bec_density = np.zeros((lattice_size, lattice_size))
    dist_from_bec_center_sq = (x - BEC_POS[0])**2 + (y - BEC_POS[1])**2
    bec_density[dist_from_bec_center_sq < BEC_RADIUS**2] = 1.0

    # The BEC wave function is complex: Psi = sqrt(density) * exp(i * phase)
    psi = np.sqrt(bec_density).astype(np.complex128)

    # Add vortices by manipulating the phase
    # Place vortices randomly within the BEC radius
    vortex_positions = []
    for _ in range(NUM_VORTICES):
        r = np.random.uniform(0, BEC_RADIUS * 0.8)
        theta = np.random.uniform(0, 2 * np.pi)
        vx = BEC_POS[0] + r * np.cos(theta)
        vy = BEC_POS[1] + r * np.sin(theta)
        vortex_positions.append((vx, vy))

        # Add the phase contribution of this vortex
        phase = VORTEX_CHARGE * np.arctan2(y - vy, x - vx)
        psi *= np.exp(1j * phase)

    print(f"✅ BEC initialized with {NUM_VORTICES} quantum vortices.")
    return psi, vortex_positions

def get_vortex_info_field(psi):
    """
    Calculates the NGFT Information Field from the BEC vortices.
    In NGFT, information is related to complexity and gradients. We model this
    as being high where the BEC phase changes rapidly (i.e., near the vortex cores).
    """
    # Phase gradient magnitude is a good proxy for informational complexity
    phase = np.angle(psi)
    grad_phase_y, grad_phase_x = np.gradient(phase)
    info_field = np.sqrt(grad_phase_x**2 + grad_phase_y**2)

    # Normalize and clip to prevent extreme values at the vortex singularity
    info_field /= np.max(info_field) if np.max(info_field) > 0 else 1.0
    return gaussian_filter(info_field, sigma=VORTEX_CORE_SIZE)

def get_net_potential(gravity_potential, vortex_info_field):
    """
    Calculates the net potential by combining gravity (attractive)
    and the NGFT vortex effect (repulsive).
    """
    # The "programmed" vortices create a repulsive potential hill
    vortex_repulsion = VORTEX_REPULSION_STRENGTH * vortex_info_field

    # The net potential is the sum of the two
    return GRAVITATIONAL_STRENGTH * gravity_potential + vortex_repulsion

def evolve_particle(particle, net_potential, dt):
    """Evolves the test particle's motion based on the gradient of the net potential."""
    # The force is the negative gradient of the potential field
    grad_pot_y, grad_pot_x = np.gradient(net_potential)

    # Interpolate the force at the particle's current position
    ix, iy = int(particle.x), int(particle.y)
    if not (0 <= ix < LATTICE_SIZE-1 and 0 <= iy < LATTICE_SIZE-1):
        return # Particle is off-grid

    force_x = -grad_pot_x[iy, ix]
    force_y = -grad_pot_y[iy, ix]

    # Update velocity (F=ma, so a=F/m, let m=1)
    particle.vx += force_x * dt
    particle.vy += force_y * dt

    # Update position
    particle.x += particle.vx * dt
    particle.y += particle.vy * dt

    particle.path.append((particle.x, particle.y))

print("✅ Cell 3 Complete (V23): Core functions for vortex simulation defined.")

✅ Cell 3 Complete (V23): Core functions for vortex simulation defined.


In [5]:
# Cell 4: Main Simulation Loop, Execution, and Analysis (V23)

def run_vortex_simulation():
    # --- Step 1: Initialization ---
    print("🚀 Initializing NGFT Quantum Vortex simulation...")
    gravity_potential = initialize_gravity_field(LATTICE_SIZE)
    bec_psi, vortex_positions = initialize_bec_with_vortices(LATTICE_SIZE)
    test_particle = TestParticle(TEST_PARTICLE_START_POS[0], TEST_PARTICLE_START_POS[1])

    # --- Step 2: Matplotlib Setup ---
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_facecolor('black')
    ax.set_aspect('equal')
    ax.set_xlim(0, LATTICE_SIZE); ax.set_ylim(0, LATTICE_SIZE)
    ax.tick_params(labelcolor='white', color='white')
    for spine in ax.spines.values(): spine.set_edgecolor('white')

    # Calculate the net potential for visualization
    vortex_info_field = get_vortex_info_field(bec_psi)
    net_potential = get_net_potential(gravity_potential, vortex_info_field)

    # Plot the potential field
    potential_display = ax.imshow(net_potential, cmap='coolwarm_r', origin='lower', animated=True)
    fig.colorbar(potential_display, ax=ax, label="Net Potential (Blue=Gravity, Red=NGFT Repulsion)")

    # Plot objects
    ax.add_patch(plt.Circle(MASSIVE_OBJECT_POS, MASSIVE_OBJECT_RADIUS, color='blue', alpha=0.3, label='Gravitational Well'))
    ax.add_patch(plt.Circle(BEC_POS, BEC_RADIUS, ec='cyan', fc='none', lw=2, ls='--', label='BEC Boundary'))
    ax.plot(*zip(*vortex_positions), 'c^', markersize=8, label='Quantum Vortices')
    particle_plot, = ax.plot([], [], 'o', color='lime', markersize=8, label='Test Particle')
    path_plot, = ax.plot([], [], '-', color='lime', alpha=0.5, lw=1.5)

    ax.legend(loc='upper right', facecolor='gray', framealpha=0.8)
    main_title = ax.set_title("", color='white', fontsize=16)

    # --- Step 3: Animation Loop ---
    def animate(frame):
        # In a more complex simulation, the BEC itself would evolve. Here, we keep it static.
        # We re-calculate the net potential in case we make it dynamic later.
        vortex_info_field = get_vortex_info_field(bec_psi)
        net_potential = get_net_potential(gravity_potential, vortex_info_field)

        # Evolve the test particle
        evolve_particle(test_particle, net_potential, DT)

        # Update plots
        particle_plot.set_data([test_particle.x], [test_particle.y])
        path_x, path_y = zip(*test_particle.path)
        path_plot.set_data(path_x, path_y)

        main_title.set_text(f"NGFT Vortex Gravity Mitigation (V23)\nFrame: {frame}/{TOTAL_FRAMES}")
        return [particle_plot, path_plot, main_title]

    # --- Step 4: Run Animation and Analysis ---
    print("Starting V23 simulation...")
    ani = animation.FuncAnimation(fig, animate, frames=TOTAL_FRAMES, interval=40, blit=True, repeat=False)

    # Save the animation as a GIF
    gif_filename = "vortex_simulation.gif"
    print(f"Saving animation to {gif_filename}...")
    ani.save(gif_filename, writer='pillow', fps=25) # Use 'pillow' writer for GIF

    plt.close(fig)
    print(f"✅ Animation saved to {gif_filename}")

    print("\n--- FINAL ANALYSIS OF SIMULATION RESULTS (V23) ---")

    final_pos = test_particle.path[-1]
    dist_to_center = np.sqrt((final_pos[0]-MASSIVE_OBJECT_POS[0])**2 + (final_pos[1]-MASSIVE_OBJECT_POS[1])**2)

    print(f"Final particle distance from massive object center: {dist_to_center:.2f} units.")
    if dist_to_center > MASSIVE_OBJECT_RADIUS * 1.5:
        print("\n✅ SUCCESS: The particle was significantly deflected or repelled.")
        print("This indicates that the NGFT Quantum Vortices successfully generated a 'counter-gravity' or repulsive")
        print("potential, mitigating the gravitational pull of the massive object.")
    else:
        print("\n⚠️ NOTE: The particle was not significantly deflected. It fell into the gravity well.")
        print("Consider increasing VORTEX_REPULSION_STRENGTH in Cell 2.")

# Execute the main function
run_vortex_simulation()

🚀 Initializing NGFT Quantum Vortex simulation...
✅ BEC initialized with 7 quantum vortices.
Starting V23 simulation...
Saving animation to vortex_simulation.gif...
✅ Animation saved to vortex_simulation.gif

--- FINAL ANALYSIS OF SIMULATION RESULTS (V23) ---
Final particle distance from massive object center: 49.90 units.

✅ SUCCESS: The particle was significantly deflected or repelled.
This indicates that the NGFT Quantum Vortices successfully generated a 'counter-gravity' or repulsive
potential, mitigating the gravitational pull of the massive object.
